<h1>Project Course</h1>
This notebook is used for the final test of IBM course

In [1]:
import pandas as pd
import numpy as np

In [2]:
print ('Hello Capstone Project Course')

Hello Capstone Project Course


<h2> Part 2 </h2><br>
Here we gonna get the table of postal codes of Canada in a Wikipedia page using the libraries BeautifulSoup and urlib3. <br><br>
After, we gonna input this data in a DataFrame and make some arrangements

In [3]:
import urllib3
from bs4 import BeautifulSoup
urllib3.disable_warnings()
http = urllib3.poolmanager.PoolManager()
html_doc = http.urlopen('GET', 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
html_doc = html_doc.data.decode('utf-8')
soup = BeautifulSoup(html_doc, 'html.parser')
table = soup.table.get_text()
table = table.split('\n')
j = 0
for key, i in enumerate(table):
    if i == '':
        del table[key]      
for key, i in enumerate(table):
    if i == '':
        del table[key]
rows = int(len(table)/3)
table = np.reshape(table,(rows,3))
df = pd.DataFrame(table)
df.columns = df.iloc[0]
df = df.iloc[1:]
df = df[df.Borough != 'Not assigned']
df = df.groupby(["Postcode",'Borough'])['Neighbourhood'].apply(lambda tags: ','.join(tags))
df = df.reset_index()
df['Neighbourhood'] = np.where(df['Neighbourhood']=='Not assigned', df['Borough'], df['Neighbourhood'])
df.shape

(103, 3)

<h2>Part 3</h2>

Setting the latitude and longitude. <br>
The geocoder is not working.

In [4]:
!wget -q -O 'Geospatial_data.csv' https://cocl.us/Geospatial_data

In [5]:
df2 = pd.read_csv('Geospatial_data.csv')
df2 = df2.rename(index=str, columns={'Postal Code':'Postcode'})
df2.head()
df = pd.merge(df,
                df2[['Postcode','Latitude', 'Longitude']],
               on='Postcode')
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


.

In [9]:
!conda install -c conda-forge folium
import folium


Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.8.3-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  53.10 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  34.20 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  38.76 MB/s
folium-0.8.3-p 100% |################################| Time: 0:00:00  48.87 MB/s


In [10]:
latitude = 43.653908
longitude = -79.239476
map_canada = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_canada)  
    
map_canada

In [11]:
CLIENT_ID = 'SYLSO3XDBXIPU2EBQ2OBDAJ4DLTW0IRVTLVKI55XBA5LKHNC' # your Foursquare ID
CLIENT_SECRET = '1JXE5YUQHGNKO0KJ5KKTYMADOQEA0VX2U1NNI14ZKA3ATTQT' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [12]:
toronto_data = df[df['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [28]:
from geopy.geocoders import Nominatim
addresses = set(toronto_data['Borough'].tolist())
addresses

{'Central Toronto', 'Downtown Toronto', 'East Toronto', 'West Toronto'}

In [29]:

for address in addresses:
    geolocator = Nominatim(user_agent="ny_explorer")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    print('The geograpical coordinate of {} are {}, {}.'.format(address,latitude, longitude))

The geograpical coordinate of East Toronto are 43.653963, -79.387207.
The geograpical coordinate of West Toronto are 43.653963, -79.387207.
The geograpical coordinate of Downtown Toronto are 43.6541737, -79.3808116451341.
The geograpical coordinate of Central Toronto are 43.653963, -79.387207.


In [53]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [17]:
search_query = ''
radius = 500
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=SYLSO3XDBXIPU2EBQ2OBDAJ4DLTW0IRVTLVKI55XBA5LKHNC&client_secret=1JXE5YUQHGNKO0KJ5KKTYMADOQEA0VX2U1NNI14ZKA3ATTQT&v=20180605&ll=43.67635739999999,-79.2930312&radius=500&limit=100'

In [19]:
import requests
results = requests.get(url).json()

In [20]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [54]:
from pandas.io.json import json_normalize

In [56]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']

        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [57]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )


In [60]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")



toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]


In [61]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

In [62]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [63]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

In [64]:
kclusters = 5
from sklearn.cluster import KMeans
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

In [65]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data
toronto_merged = toronto_merged.rename(index=str, columns={'Neighbourhood':'Neighborhood'})
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

In [66]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
import matplotlib.cm as cm
import matplotlib.colors as colors
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters